# GEH Pharmacy Department

In [1]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import *

## Merge April & May Data

In [2]:
files_xls_1 = ['../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT (April 2021).xlsx', '../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT 2021 May.xlsx']

In [3]:
df = pd.DataFrame()

In [4]:
for f in files_xls_1:
    data = pd.read_excel(f, 'Template')
    df = df.append(data)

In [5]:
df

,Verifier/Typist,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Checker,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,Date,Case No,Room No,Ward,SCM / Handwritten TTO,No. of items,Time in / Verified,Time checked,Checked by,Time to meet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-01 00:00:00,1021023186,957,9W,SCM,2,08:59:00,09:04:00,LET,09:44:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-01 00:00:00,1021023191,509,5E,SCM,3,09:05:00,09:26:00,LET,09:50:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-01 00:00:00,1021023190,508,5E,SCM,4,09:07:00,09:19:00,LET,09:52:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-01 00:00:00,1021023367,932,9E,SCM,2,09:11:00,09:22:00,ELISE,09:56:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.columns = df.iloc[0]

In [7]:
df = df.drop(0)
df

,Date,Case No,Room No,Ward,SCM / Handwritten TTO,No. of items,Time in / Verified,Time checked,Checked by,Time to meet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-01 00:00:00,1021023186,957,9W,SCM,2,08:59:00,09:04:00,LET,09:44:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-01 00:00:00,1021023191,509,5E,SCM,3,09:05:00,09:26:00,LET,09:50:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-01 00:00:00,1021023190,508,5E,SCM,4,09:07:00,09:19:00,LET,09:52:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-01 00:00:00,1021023367,932,9E,SCM,2,09:11:00,09:22:00,ELISE,09:56:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-04-01 00:00:00,1021023455,909,9E,SCM,5,09:36:00,10:02:00,ELISE,10:21:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df = df.dropna(axis=1, how='all')
df

,Date,Case No,Room No,Ward,SCM / Handwritten TTO,No. of items,Time in / Verified,Time checked,Checked by,Time to meet,Time reached,Time Counselled,Counselled by,Remarks,Limo Service,Timings (Checked),Timings (Reached),NaN
1,2021-04-01 00:00:00,1021023186,957,9W,SCM,2,08:59:00,09:04:00,LET,09:44:00,09:29:00,09:35:00,LET,NaN,NaN,00:05:00,00:30:00,NaN
2,2021-04-01 00:00:00,1021023191,509,5E,SCM,3,09:05:00,09:26:00,LET,09:50:00,09:29:00,09:32:00,ELISE,NaN,NaN,00:21:00,00:24:00,NaN
3,2021-04-01 00:00:00,1021023190,508,5E,SCM,4,09:07:00,09:19:00,LET,09:52:00,09:32:00,10:17:00,ELISE,P,NaN,00:12:00,00:25:00,NaN
4,2021-04-01 00:00:00,1021023367,932,9E,SCM,2,09:11:00,09:22:00,ELISE,09:56:00,09:37:00,09:40:00,LET,NaN,NaN,00:11:00,00:26:00,NaN
5,2021-04-01 00:00:00,1021023455,909,9E,SCM,5,09:36:00,10:02:00,ELISE,10:21:00,10:21:00,10:25:00,ELISE,NaN,NaN,00:26:00,00:45:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df = df[pd.notnull(df['Case No'])]

In [10]:
df

,Date,Case No,Room No,Ward,SCM / Handwritten TTO,No. of items,Time in / Verified,Time checked,Checked by,Time to meet,Time reached,Time Counselled,Counselled by,Remarks,Limo Service,Timings (Checked),Timings (Reached),NaN
1,2021-04-01 00:00:00,1021023186,957,9W,SCM,2,08:59:00,09:04:00,LET,09:44:00,09:29:00,09:35:00,LET,NaN,NaN,00:05:00,00:30:00,NaN
2,2021-04-01 00:00:00,1021023191,509,5E,SCM,3,09:05:00,09:26:00,LET,09:50:00,09:29:00,09:32:00,ELISE,NaN,NaN,00:21:00,00:24:00,NaN
3,2021-04-01 00:00:00,1021023190,508,5E,SCM,4,09:07:00,09:19:00,LET,09:52:00,09:32:00,10:17:00,ELISE,P,NaN,00:12:00,00:25:00,NaN
4,2021-04-01 00:00:00,1021023367,932,9E,SCM,2,09:11:00,09:22:00,ELISE,09:56:00,09:37:00,09:40:00,LET,NaN,NaN,00:11:00,00:26:00,NaN
5,2021-04-01 00:00:00,1021023455,909,9E,SCM,5,09:36:00,10:02:00,ELISE,10:21:00,10:21:00,10:25:00,ELISE,NaN,NaN,00:26:00,00:45:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,NaN,1021037723,112,10W,SCM,2,09:24:00,10:53:00,TEL,10:09:00,11:10:00,11:17:00,TEL,patient shower,NaN,01:29:00,01:46:00,NaN
204,NaN,1021037197,921,9E,SCM,1,10:38:00,10:53:00,TEL,11:23:00,11:15:00,NaN,TEL,Patient not available,NaN,00:15:00,00:37:00,NaN
205,NaN,1021037427,101,10W,SCM,1,12:00:00,12:05:00,TEL,12:45:00,12:10:00,12:20:00,TEL,NaN,NaN,00:05:00,00:10:00,NaN
206,NaN,1021039189,928,9E,scm,3,12:57:00,13:22:00,WQ,13:42:00,13:40:00,13:47:00,TEL,NaN,NaN,00:25:00,00:43:00,NaN


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 1 to 207
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Date                    424 non-null    object
 1   Case No                 497 non-null    object
 2   Room No                 497 non-null    object
 3   Ward                    497 non-null    object
 4   SCM / Handwritten TTO   497 non-null    object
 5   No. of items            497 non-null    object
 6   Time in / Verified      491 non-null    object
 7   Time checked            471 non-null    object
 8   Checked by              471 non-null    object
 9   Time to meet            497 non-null    object
 10  Time reached            434 non-null    object
 11   Time Counselled        390 non-null    object
 12  Counselled by           423 non-null    object
 13  Remarks                 154 non-null    object
 14  Limo Service            1 non-null      object
 15  Timing

### Drop columns

In [12]:
df2= df.copy()

In [13]:
df2 = df2.drop(df.iloc[:, 17:], axis=1)

In [14]:
#reset index 
df2 = df2.reset_index(drop=True)

### Rename common columns

In [15]:
df2.rename({'Date ': 'Date', 
            'Case No': 'Case Number', 
            'Room No': 'Room Number',
            'SCM / Handwritten TTO ': 'TTO Type', 
            'No. of items':'No. of Drugs', 
            'Time in / Verified': 'Time TTO Received', # the start time
            'Time checked':'Time TTO Checked', 
            'Time reached ': 'Time TTO Reached', # the end time 
            ' Time Counselled':'Time TTO Counselled', # end time if patients need to be counselled
           'Limo Service ':'Limo Service'},  
           axis=1, inplace=True)

In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Date                 424 non-null    object
 1   Case Number          497 non-null    object
 2   Room Number          497 non-null    object
 3   Ward                 497 non-null    object
 4   TTO Type             497 non-null    object
 5   No. of Drugs         497 non-null    object
 6   Time TTO Received    491 non-null    object
 7   Time TTO Checked     471 non-null    object
 8   Checked by           471 non-null    object
 9   Time to meet         497 non-null    object
 10  Time TTO Reached     434 non-null    object
 11  Time TTO Counselled  390 non-null    object
 12  Counselled by        423 non-null    object
 13  Remarks              154 non-null    object
 14  Limo Service         1 non-null      object
 15  Timings (Checked)    489 non-null    object
 16  Timings 

### Handle missing value

In [17]:
list(df2.columns)

['Date',
 'Case Number',
 'Room Number',
 'Ward',
 'TTO Type',
 'No. of Drugs',
 'Time TTO Received',
 'Time TTO Checked',
 'Checked by',
 'Time to meet',
 'Time TTO Reached',
 'Time TTO Counselled',
 'Counselled by',
 'Remarks',
 'Limo Service',
 'Timings (Checked)',
 'Timings (Reached)']

In [18]:
df2.isnull().sum()

0
Date                    73
Case Number              0
Room Number              0
Ward                     0
TTO Type                 0
No. of Drugs             0
Time TTO Received        6
Time TTO Checked        26
Checked by              26
Time to meet             0
Time TTO Reached        63
Time TTO Counselled    107
Counselled by           74
Remarks                343
Limo Service           496
Timings (Checked)        8
Timings (Reached)       11
dtype: int64

#### Missing date

In [19]:
# change to datetime formate
df2['Date'] = pd.to_datetime(df2['Date'])

In [20]:
missing_date = df2[df2['Date'].isnull()]
missing_date

,Date,Case Number,Room Number,Ward,TTO Type,No. of Drugs,Time TTO Received,Time TTO Checked,Checked by,Time to meet,Time TTO Reached,Time TTO Counselled,Counselled by,Remarks,Limo Service,Timings (Checked),Timings (Reached)
386,NaT,1021035404,609,6E,SCM,3,09:10:00,09:22:00,HEMIN,09:55:00,09:52:00,09:55:00,HEMIN,NaN,NaN,00:12:00,00:42:00
387,NaT,1021033687,919,9E,SCM,6,09:21:00,09:39:00,HEMIN,10:06:00,11:02:00,11:09:00,LJY,NaN,NaN,00:18:00,01:41:00
388,NaT,1021032404,228,10W,SCM,3,09:33:00,10:10:00,LJY,10:18:00,10:40:00,10:44:00,HEMIN,NaN,NaN,00:37:00,01:07:00
389,NaT,1021034765,916,9E,SCM,4,09:35:00,09:43:00,HEMIN,10:20:00,10:00:00,10:04:00,AKH,NaN,NaN,00:08:00,00:25:00
390,NaT,1021035355,755,7W,SCM,2,09:41:00,09:51:00,LJY,10:26:00,NaN,NaN,NaN,WAIT FOR SON,NaN,00:10:00,1899-12-30 14:19:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,NaT,1021037723,112,10W,SCM,2,09:24:00,10:53:00,TEL,10:09:00,11:10:00,11:17:00,TEL,patient shower,NaN,01:29:00,01:46:00
493,NaT,1021037197,921,9E,SCM,1,10:38:00,10:53:00,TEL,11:23:00,11:15:00,NaN,TEL,Patient not available,NaN,00:15:00,00:37:00
494,NaT,1021037427,101,10W,SCM,1,12:00:00,12:05:00,TEL,12:45:00,12:10:00,12:20:00,TEL,NaN,NaN,00:05:00,00:10:00
495,NaT,1021039189,928,9E,scm,3,12:57:00,13:22:00,WQ,13:42:00,13:40:00,13:47:00,TEL,NaN,NaN,00:25:00,00:43:00


In [21]:
# fill up missing date based on previous known value 
df2['Date'] = df2['Date'].fillna(method='ffill')
df2.tail()

,Date,Case Number,Room Number,Ward,TTO Type,No. of Drugs,Time TTO Received,Time TTO Checked,Checked by,Time to meet,Time TTO Reached,Time TTO Counselled,Counselled by,Remarks,Limo Service,Timings (Checked),Timings (Reached)
492,2021-05-31,1021037723,112,10W,SCM,2,09:24:00,10:53:00,TEL,10:09:00,11:10:00,11:17:00,TEL,patient shower,NaN,01:29:00,01:46:00
493,2021-05-31,1021037197,921,9E,SCM,1,10:38:00,10:53:00,TEL,11:23:00,11:15:00,NaN,TEL,Patient not available,NaN,00:15:00,00:37:00
494,2021-05-31,1021037427,101,10W,SCM,1,12:00:00,12:05:00,TEL,12:45:00,12:10:00,12:20:00,TEL,NaN,NaN,00:05:00,00:10:00
495,2021-05-31,1021039189,928,9E,scm,3,12:57:00,13:22:00,WQ,13:42:00,13:40:00,13:47:00,TEL,NaN,NaN,00:25:00,00:43:00
496,2021-05-31,1021038490,622,TU,SCM,13,11:05:00,13:15:00,WQ,11:50:00,13:25:00,13:39:00,TEL,waiting for clinic TTO,NaN,02:10:00,02:20:00


#### Missing Time TTO Received

In [22]:
# drop the one missing row in april dataset bc no time in cant calculate time taken
# drop row in may too for same reason
df2 = df2.dropna(subset=['Time TTO Received'])

In [23]:
df2.isnull().sum()

0
Date                     0
Case Number              0
Room Number              0
Ward                     0
TTO Type                 0
No. of Drugs             0
Time TTO Received        0
Time TTO Checked        24
Checked by              25
Time to meet             0
Time TTO Reached        60
Time TTO Counselled    104
Counselled by           71
Remarks                340
Limo Service           490
Timings (Checked)        8
Timings (Reached)       11
dtype: int64

### EDA

In [24]:
df2

,Date,Case Number,Room Number,Ward,TTO Type,No. of Drugs,Time TTO Received,Time TTO Checked,Checked by,Time to meet,Time TTO Reached,Time TTO Counselled,Counselled by,Remarks,Limo Service,Timings (Checked),Timings (Reached)
0,2021-04-01,1021023186,957,9W,SCM,2,08:59:00,09:04:00,LET,09:44:00,09:29:00,09:35:00,LET,NaN,NaN,00:05:00,00:30:00
1,2021-04-01,1021023191,509,5E,SCM,3,09:05:00,09:26:00,LET,09:50:00,09:29:00,09:32:00,ELISE,NaN,NaN,00:21:00,00:24:00
2,2021-04-01,1021023190,508,5E,SCM,4,09:07:00,09:19:00,LET,09:52:00,09:32:00,10:17:00,ELISE,P,NaN,00:12:00,00:25:00
3,2021-04-01,1021023367,932,9E,SCM,2,09:11:00,09:22:00,ELISE,09:56:00,09:37:00,09:40:00,LET,NaN,NaN,00:11:00,00:26:00
4,2021-04-01,1021023455,909,9E,SCM,5,09:36:00,10:02:00,ELISE,10:21:00,10:21:00,10:25:00,ELISE,NaN,NaN,00:26:00,00:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,2021-05-31,1021037723,112,10W,SCM,2,09:24:00,10:53:00,TEL,10:09:00,11:10:00,11:17:00,TEL,patient shower,NaN,01:29:00,01:46:00
493,2021-05-31,1021037197,921,9E,SCM,1,10:38:00,10:53:00,TEL,11:23:00,11:15:00,NaN,TEL,Patient not available,NaN,00:15:00,00:37:00
494,2021-05-31,1021037427,101,10W,SCM,1,12:00:00,12:05:00,TEL,12:45:00,12:10:00,12:20:00,TEL,NaN,NaN,00:05:00,00:10:00
495,2021-05-31,1021039189,928,9E,scm,3,12:57:00,13:22:00,WQ,13:42:00,13:40:00,13:47:00,TEL,NaN,NaN,00:25:00,00:43:00


#### Day of Week

In [25]:
df2['Weekday'] = df2['Date'].dt.day_name()

In [26]:
# handle Unknown string format: 9.:09 in Time TTO reached
df2['Time TTO Reached'] = df2['Time TTO Reached'].replace('9.:09', 'NA')

In [27]:
#tto_received = pd.to_datetime(df2['Time TTO Received'].astype(str)) 
#tto_reached = pd.to_datetime(df2[df2['Time TTO Reached'] != 'NA']['Time TTO Reached'].astype(str))
#tto_counselled = pd.to_datetime(df2[df2['Time TTO Counselled'] != 'NA']['Time TTO Counselled'].astype(str))

#df2['Overall Time Taken (mins)'] = tto_reached.sub(tto_received).dt.total_seconds().div(60)

In [28]:
#df2.info()

In [29]:
#df2['time_diff'] = pd.Timestamp(df2['Time TTO Received']) - pd.Timestamp(df['end_time'])   
#def f(row):
#    if row['Time TTO Counselled'] != " ":
#        val = tto_counselled.sub(tto_received).dt.total_seconds().div(60)
    #elif row['Time TTO Counselled'] <  row['Time TTO Reached']:
    #    val = tto_reached.sub(tto_received).dt.total_seconds().div(60)
#    else:
#        val = tto_reached.sub(tto_received).dt.total_seconds().div(60)
#    return val

In [30]:
#df2['Overall Time Taken (mins)_2'] = df2.apply(f, axis=1)
#df2

### Export to CSV for Final Merge

In [31]:
df2.to_csv("../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT (April-May 2021).csv", index = False, header=True)

## Merge remaining months

In [32]:
files_xls_2 = ['../GEH TTO Data/GEH TTO TAT 2021 June.xlsx',
 '../GEH TTO Data/GEH TTO TAT 2021 July.xlsx',
 '../GEH TTO Data/GEH TTO TAT 2021 Aug.xlsx',
'../GEH TTO Data/GEH TTO TAT 2021 Sep.xlsx',
 '../GEH TTO Data/GEH TTO TAT 2021 Nov.xlsx', 
'../GEH TTO Data/GEH TTO TAT 2021 Oct.xlsx', 
 '../GEH TTO Data/GEH TTO TAT 2021 Dec.xlsx']

In [33]:
df3 = pd.DataFrame()
for f in files_xls_2:
    data = pd.read_excel(f)
    df3 = df3.append(data)
df3

FileNotFoundError: [Errno 2] No such file or directory: '../GEH TTO Data/GEH TTO TAT 2021 June.xlsx'

In [ ]:
df3.info()

In [ ]:
#reset index 
df3 = df3.reset_index(drop=True)

### Drop columns

In [ ]:
# drop columns that should be ignored
df3 = df3.drop(['TTO_TAT_Calc', 'Total number of TTO', 'Status_Time_Met', 'ID', 'Title', 'TTOaddon', 'Item Type', 'Path', 'Unnamed: 23', 'Unnamed: 24'], axis = 1)
# drop other redundant columns
#df3 = df3.drop(['Pharmacist_Checked', 'Pharmacist_Dispensed', 'Dispatcher' ], axis = 1)

### Rename common columns

In [ ]:
df3.info()

In [ ]:
df3.rename({'Room': 'Room Number',
            'No. of Items':'No. of Drugs', 
            'Time In': 'Time TTO Received', # the start time
            'Time Checked':'Time TTO Checked', 
            'Time Reached': 'Time TTO Reached', # the end time 
            'Time Dispensed' : 'Time TTO Counselled'},
           axis=1, inplace=True)

In [ ]:
columns = ['Time TTO Received', 'Time TTO Checked', 'Time TTO Reached', 'Time TTO Counselled']

for column in columns:
    # remove date in the time metric
    #df3[column]=df3[column].dt.time 
    df3[column] = df3[column].astype(str).str.split(' ').str[-1]

In [ ]:
df3

In [ ]:
df3.info()

### Handle missing value

In [ ]:
df3.isnull().sum()

#### Missing Date

In [ ]:
# change to datetime format
df3['Date'] = pd.to_datetime(df3['Date'])

In [ ]:
missing_date = df3[df3['Date'].isnull()]
missing_date

In [ ]:
#drop rows
df3 = df3.dropna(subset=['Date'])

In [ ]:
# missing pharmacist dispensed
missing_date = df3[df3['Pharmacist_Dispensed'].isnull()]
missing_date.head()

In [ ]:
df3['No. of Drugs'].replace(np.nan, 'NA', inplace = True)

In [ ]:
missing_date = df3[df3['Time'].notnull()]
missing_date

In [ ]:
df3.isnull().sum()

### EDA

#### Day of Week

In [ ]:
# add weekday column
df3['Weekday'] = df3['Date'].dt.day_name()

### Export to CSV for Final Merge

In [ ]:
df3.to_csv("../../Data/Pharmacy Dept/GEH TTO Data/GEH TTO TAT (June-Dec 2021).csv", index = None, header=True)

## Final Merge of the above CSVs

In [ ]:
merged_df = pd.concat([df2, df3])

In [ ]:
merged_df

In [ ]:
merged_df['Date'] = pd.to_datetime(merged_df['Date']).dt.normalize()

In [ ]:
merged_df.head()

In [ ]:
list(merged_df.columns)

In [ ]:
merged_df = merged_df.drop(['Pharmacist_Checked','Pharmacist_Dispensed','Dispatcher','status_CheckedTTO','status_TTOReached','status_Dispensed','status_TTOIn','Time',
                           'Timings (Checked)','Timings (Reached)'],
                           axis=1)

In [ ]:
merged_df.info()

In [ ]:
merged_df[merged_df.index.duplicated()]

In [ ]:
#reset index 
merged_df = merged_df.reset_index(drop=True)

In [ ]:
merged_df

#### Overall Time Taken (mins)

In [ ]:
merged_df["Time TTO Reached"] = pd.to_datetime(merged_df[merged_df['Time TTO Reached'] != 'NA']['Time TTO Reached'].astype(str))
merged_df["Time TTO Counselled"]= pd.to_datetime(merged_df[merged_df['Time TTO Counselled'] != 'NA']['Time TTO Counselled'].astype(str))

In [ ]:
merged_df["TTO Last Timestamp"] = merged_df[['Time TTO Reached', 'Time TTO Counselled']].max(axis=1)
#merged_df["Last Timestamp"]  = merged_df['Last Timestamp'].dt.time

In [ ]:
#columns = ['Time TTO Reached', 'Time TTO Counselled', 'Last Timestamp']

#for column in column:
#    merged_df[column]  = merged_df[column].dt.time

In [ ]:
#merged_df["Time TTO Received"]  = pd.to_datetime(merged_df["Time TTO Received"] , infer_datetime_format=True, errors='coerce')
#merged_df["Time TTO Reached"]  = pd.to_datetime(merged_df["Time TTO Reached"] , infer_datetime_format=True, errors='coerce')
#merged_df["Time TTO Counselled"]  = pd.to_datetime(merged_df["Time TTO Counselled"] , infer_datetime_format=True, errors='coerce')

In [ ]:
merged_df["Time TTO Reached"]  = merged_df['Time TTO Reached'].dt.time
merged_df["Time TTO Counselled"]  = merged_df['Time TTO Counselled'].dt.time

In [ ]:
merged_df["TTO Last Timestamp"]  = merged_df['TTO Last Timestamp'].dt.time

In [ ]:
merged_df.info()

In [ ]:
merged_df

In [ ]:
tto_received = pd.to_datetime(merged_df['Time TTO Received'].astype(str)) 
tto_last_pt = pd.to_datetime(merged_df[merged_df['TTO Last Timestamp'] != 'NA']['TTO Last Timestamp'].astype(str))
#tto_counselled = pd.to_datetime(df2[df2['Time TTO Counselled'] != 'NA']['Time TTO Counselled'].astype(str))

merged_df['Overall Time Taken (mins)'] = tto_last_pt.sub(tto_received).dt.total_seconds().div(60)

In [ ]:
merged_df

#### Meet KPI

In [ ]:
def f(row):
    if row['Overall Time Taken (mins)'] > 45:
        val = 'No'
    elif row['Overall Time Taken (mins)'] > 0 and row['Overall Time Taken (mins)'] <= 45:
        val = 'Yes'
    else:
        val = 'NA'
    return val

In [ ]:
merged_df['Meet KPI'] = merged_df.apply(f, axis=1)
merged_df

In [ ]:
merged_df.info()

#### Office Hour

In [ ]:
x = datetime.strptime('08:30:00', '%H:%M:%S').time()
x

In [ ]:
merged_df["Time TTO Received"]  = pd.to_datetime(merged_df["Time TTO Received"].astype(str))
merged_df["TTO Last Timestamp"]  = pd.to_datetime(merged_df["TTO Last Timestamp"].astype(str))
#merged_df["TTO Last Timestamp"]  = pd.to_datetime(merged_df["TTO Last Timestamp"] , infer_datetime_format=True, errors='coerce')

In [ ]:
merged_df

In [ ]:
merged_df["Time TTO Received"]   = merged_df["Time TTO Received"].dt.time
merged_df["TTO Last Timestamp"]   = merged_df["TTO Last Timestamp"].dt.time

In [ ]:
merged_df['Office Hours'] = np.where(merged_df['Time TTO Received'].isnull() | merged_df['TTO Last Timestamp'].isnull() , 'NA', 
                            (np.where(( #(df['Time TTO Received'] >= pd.to_datetime('08:00:00').time()) & 
                                        (merged_df['TTO Last Timestamp'] <= pd.to_datetime('20:00:00').time())),
                                        'Yes', 'No')))

merged_df

### Export Final CLEANED to csv

In [ ]:
merged_df.to_csv("../../Data/Pharmacy Dept/Data Cleaning/GEH_Data_Cleaned.csv", index = None, header=True)

In [ ]:
merged_df.isnull().sum()

## EDA

In [ ]:
df = pd.read_csv("../../Data/Pharmacy Dept/Data Cleaning/GEH_Data_Cleaned.csv")

In [ ]:
df.notnull().sum()

In [ ]:
df.isnull().sum()

### Average Prescriptions Filled Per Month

In [ ]:
#df['Date'] = pd.to_datetime(df['Date'])
#df.groupby(df['Date'].dt.strftime('%B'))['No. of Drugs'].sum().sort_values()

In [ ]:
#df.groupby(df['Date'].dt.strftime('%B'))['No. of Items'].mean().sort_values()